In [1]:
import torch
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim, nn

In [2]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = datasets.FashionMNIST("fashin_data1/", train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)

testset = datasets.FashionMNIST("fashion_data1/", train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

In [48]:
class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        super().__init__()
        # Input to a hidden layer
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        for each in self.hidden_layers:
            x = F.relu(each(x))
            x = self.dropout(x)
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)

In [49]:
def train(model, trainloader, testloader, criterion, optimizer, epochs = 5):
    
    for e in range(epochs):
        running_loss = 0
        for images, labels in trainloader:
            images = images.view(images.shape[0], -1)
            
            log_pbs = model.forward(images)
            
            optimizer.zero_grad()
            loss = criterion(log_pbs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        else:
            test_loss = 0
            accuracy = 0
            
            with torch.no_grad():
                model.eval()
                for images, labels in testloader:
                    images = images.view(images.shape[0], -1)
                    
                    log_pbs = model.forward(images)

                    test_loss += criterion(log_pbs, labels)
                    
                    ps = torch.exp(log_pbs)
                    top_p, top_class = ps.topk(1, dim = 1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            
            model.train()
            print("Epochs : {}/{}..".format(e+1, epochs),
                  "Training Loss : {:.3f}..".format(running_loss/len(trainloader)),
                  "Testing Loss : {:.3f}..".format(test_loss/len(testloader)),
                  "Accurcy : {:.3f}..".format(accuracy/len(testloader)))         

In [50]:
model = Network(784, 10, [512, 256, 128])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.003)

In [51]:
train(model, trainloader, testloader, criterion, optimizer, epochs = 3)

Epochs : 1/3.. Training Loss : 0.768.. Testing Loss : 0.518.. Accurcy : 0.814..
Epochs : 2/3.. Training Loss : 0.627.. Testing Loss : 0.484.. Accurcy : 0.828..
Epochs : 3/3.. Training Loss : 0.592.. Testing Loss : 0.511.. Accurcy : 0.807..


In [52]:
print(f"our model : {model}")
print(f"thr state dict keys : \n\n{model.state_dict().keys()}")

our model : Network(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (output): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
thr state dict keys : 

odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'output.weight', 'output.bias'])


In [53]:
checkpoint = {
                "input_size" : 784,
                "output_size" : 10,
                "hidden_layers":[each.out_features for each in model.hidden_layers],
                "state_dict": model.state_dict()
             }
torch.save(checkpoint, "checkpoint.pth")


In [56]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = Network(checkpoint["input_size"],
                   checkpoint["output_size"],
                   checkpoint["hidden_layers"])
    model.load_state_dict(checkpoint["state_dict"])
    return model

In [57]:
model = load_checkpoint('checkpoint.pth')
print(model)

Network(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (output): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
